In [ ]:
# ============================================
# 1) Upload your Excel file
# ============================================
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload your .xlsx file

excel_filename = list(uploaded.keys())[0]
print("Loaded file:", excel_filename)

df = pd.read_excel(excel_filename)
print("Columns in your file:", df.columns.tolist())
display(df.head())

# ============================================
# 2) SET THESE COLUMN NAMES CORRECTLY
# ============================================
# Change these to EXACTLY match your column names in Excel
AA_COL = 'Res_name'          # e.g. 'AA', 'AminoAcid', 'Residue', etc.
REGION_COL = 'Region'     # e.g. 'IDR', 'Region', 'IDR_region', etc.

# ============================================
# 3) Mapping: single-letter AA -> category
# ============================================

aa_category = {
    # Non-polar (hydrophobic)
    'A': "Non-polar (hydrophobic)",
    'V': "Non-polar (hydrophobic)",
    'L': "Non-polar (hydrophobic)",
    'I': "Non-polar (hydrophobic)",
    'M': "Non-polar (hydrophobic)",
    'G': "Non-polar (hydrophobic)",
    'P': "Non-polar (hydrophobic)",

    # Aromatic
    'F': "Aromatic",
    'W': "Aromatic",
    'Y': "Aromatic",

    # Polar, hydrophilic (uncharged)
    'S': "Polar, hydrophilic (uncharged)",
    'T': "Polar, hydrophilic (uncharged)",
    'N': "Polar, hydrophilic (uncharged)",
    'Q': "Polar, hydrophilic (uncharged)",
    'C': "Polar, hydrophilic (uncharged)",

    # Polar, hydrophilic (acidic)
    'D': "Polar, hydrophilic (acidic)",
    'E': "Polar, hydrophilic (acidic)",

    # Polar, hydrophilic (basic)
    'K': "Polar, hydrophilic (basic)",
    'R': "Polar, hydrophilic (basic)",
    'H': "Polar, hydrophilic (basic)",
}

# Make sure amino acid codes are uppercase single letters
df['AA_clean'] = df[AA_COL].astype(str).str.strip().str.upper()

# Map to category
df['AA_Category'] = df['AA_clean'].map(aa_category).fillna("Unknown")

print("Annotated residue-level table:")
display(df.head(20))

# ============================================
# 4) Region-wise category fractions + dominant type
# ============================================

# Drop rows with missing region
df_valid = df.dropna(subset=[REGION_COL])

# Count residues per (Region, Category)
region_cat_counts = (
    df_valid
    .groupby([REGION_COL, 'AA_Category'])
    .size()
    .reset_index(name='count')
)

print("Counts per region-category:")
display(region_cat_counts)

# Convert to fractions
region_cat_fraction = (
    region_cat_counts
    .groupby(REGION_COL)
    .apply(lambda x: x.assign(fraction=x['count'] / x['count'].sum()))
    .reset_index(drop=True)
)

print("Fractions per region-category:")
display(region_cat_fraction)

# Pivot to wide table: each IDR = one row
region_cat_table = region_cat_fraction.pivot(
    index=REGION_COL,
    columns='AA_Category',
    values='fraction'
).fillna(0.0)

# Dominant category per IDR region
region_cat_table['Dominant_Category'] = region_cat_table.idxmax(axis=1)

print("Region-level summary (IDR1–IDR5):")
display(region_cat_table)

# ============================================
# 5) Save outputs and download
# ============================================
residue_outfile = "amino_acid_with_category.xlsx"
region_outfile = "idr_region_category_summary.xlsx"

df.to_excel(residue_outfile, index=False)
region_cat_table.to_excel(region_outfile)

print("Saved files:", residue_outfile, "and", region_outfile)

from google.colab import files
files.download(residue_outfile)
files.download(region_outfile)


Saving Amino_acid_properties_checking.xlsx to Amino_acid_properties_checking (1).xlsx
Loaded file: Amino_acid_properties_checking (1).xlsx
Columns in your file: ['Res_name', 'Region']


,Res_name,Region
0,M,IDR 1
1,R,IDR 1
2,R,IDR 1
3,Y,IDR 1
4,NaN,NaN


Annotated residue-level table:


,Res_name,Region,AA_clean,AA_Category
0,M,IDR 1,M,Non-polar (hydrophobic)
1,R,IDR 1,R,"Polar, hydrophilic (basic)"
2,R,IDR 1,R,"Polar, hydrophilic (basic)"
3,Y,IDR 1,Y,Aromatic
4,NaN,NaN,NAN,Unknown
5,Q,IDR2,Q,"Polar, hydrophilic (uncharged)"
6,E,IDR2,E,"Polar, hydrophilic (acidic)"
7,P,IDR2,P,Non-polar (hydrophobic)
8,F,IDR2,F,Aromatic
9,L,IDR2,L,Non-polar (hydrophobic)


Counts per region-category:


...

Saved files: amino_acid_with_category.xlsx and idr_region_category_summary.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>